To begin, we need to transfer the device from CPU memory to the GPU.

In [1]:
import torch
device = torch.device('cuda')

Load the TRT optimized models.

In [2]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('new_best_model_trt.pth'))

<All keys matched successfully>

# Create Preprocessing Function
It's the same for both models.

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

#normalize = torchvision.transforms.Normalize(mean, std)
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

# Start & Display the Camera

In [4]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import bgr8_to_jpeg
from jetbot.camera.zmq_camera import ZmqCamera as Camera

camera = Camera()
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# Create Robot Instance
This is used to drive the motors.

In [5]:
from jetbot import Robot

robot = Robot()

# Define Sliders to Control JetBot

Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_slider): If you see JetBot is wobbling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

## Road Following Sliders

In [6]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=-0.02, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.2, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.05, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=-0.02, description='steering bias', max=0.3, min=-0.3, step=0.01)

## Collision Avoidance Sliders

In [7]:
blocked_slider = ipywidgets.widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop') 
blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.5, description='blocked threshold')

display(image_widget)
display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# Function to Run When Camera's Value Changes

In [8]:
import math

angle = 0.0
angle_last = 0.0
count_stops = 0.0
go_on = 0.0

# The number of frames to remain stopped
stop_time = 10
x = 0.0
y = 0.0
speed_value = speed_gain_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
    
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
    go_on = 1
    
    # Preprocess the image
    image_preproc = preprocess(change['new']).to(device)
    
    # Collision Avoidance Model
    prob_blocked = float(F.softmax(model_trt_collision(image_preproc), dim=1).flatten()[0])
    blocked_slider.value = prob_blocked
    stop_time = stopduration_slider.value
    if go_on == 1:
        
        # Probability threshold should be above 0.5
        if prob_blocked > blocked_threshold.value:
            print(f'Probability Blocked: {prob_blocked}')
            print("Stopped")
            count_stops += 1
            go_on = 2
            speed_gain_slider.value = 0.0
            speed_value = speed_gain_slider.value
        else:
            # Start of Road Following Model
            go_on = 1
            count_stops = 0
            xy = model_trt(image_preproc).detach().float().cpu().numpy().flatten()
            x = xy[0]
            y = (0.5 - xy[1]) / 2.0
            speed_gain_slider.value = 0.2
            speed_value = speed_gain_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            # Set x, y steering and speed to 0.0
            x = 0.0
            y = 0.0
            speed_value = 0
        else:
            go_on = 1
            count_stops = 0
            
    angle = math.atan2(x, y)
    pid = angle * steer_gain + (angle - angle_last) * steer_dgain
    steer_val = pid + steer_bias
    angle_last = angle
    robot.left_motor.value = max(min(speed_value + steer_val, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_value - steer_val, 1.0), 0.0)
    
execute({'new': camera.value})   
    

# Attach the Function to Camera for Processing

In [9]:
camera.observe(execute, names='value')

Probability Blocked: 0.54296875
Stopped
Probability Blocked: 0.794921875
Stopped
Probability Blocked: 0.73095703125
Stopped
Probability Blocked: 0.662109375
Stopped
Probability Blocked: 0.6298828125
Stopped
Probability Blocked: 0.74609375
Stopped
Probability Blocked: 0.93701171875
Stopped
Probability Blocked: 0.64013671875
Stopped
Probability Blocked: 0.398193359375
Stopped
Probability Blocked: 0.79638671875
Stopped
Probability Blocked: 0.5625
Stopped
Probability Blocked: 0.880859375
Stopped
Probability Blocked: 0.8818359375
Stopped
Probability Blocked: 0.98291015625
Stopped
Probability Blocked: 0.84814453125
Stopped
Probability Blocked: 0.51416015625
Stopped
Probability Blocked: 0.62451171875
Stopped
Probability Blocked: 0.450439453125
Stopped
Probability Blocked: 0.5400390625
Stopped
Probability Blocked: 0.88671875
Stopped
Probability Blocked: 0.74365234375
Stopped
Probability Blocked: 0.82861328125
Stopped
Probability Blocked: 0.94091796875
Stopped
Probability Blocked: 0.93994140625

# Run Cell Below to Stop

In [10]:
import time

camera.unobserve(execute, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

In [14]:
robot.stop()

Probability Blocked: 0.03521728515625
Probability Blocked: 0.0267791748046875
Probability Blocked: 0.040618896484375
Probability Blocked: 0.0518798828125
Probability Blocked: 0.0306243896484375
Probability Blocked: 0.033966064453125
Probability Blocked: 0.041656494140625
Probability Blocked: 0.056640625
Probability Blocked: 0.025390625
Probability Blocked: 0.03497314453125
Probability Blocked: 0.032379150390625
Probability Blocked: 0.0217742919921875
Probability Blocked: 0.0254974365234375
Probability Blocked: 0.07391357421875
Probability Blocked: 0.10015869140625
Probability Blocked: 0.0308837890625
Probability Blocked: 0.031524658203125
Probability Blocked: 0.039306640625
Probability Blocked: 0.0377197265625
Probability Blocked: 0.060760498046875
Probability Blocked: 0.04180908203125
Probability Blocked: 0.044677734375
Probability Blocked: 0.06597900390625
Probability Blocked: 0.041229248046875
Probability Blocked: 0.04931640625
Probability Blocked: 0.05169677734375
Probability Block